# Management Workflow 테스트 노트북

이 노트북은 Act1-Entertainment 프로젝트의 기본 구조를 따르면서 Management Workflow를 테스트합니다.

프로젝트 구조:
- `BaseWorkflow`를 상속받은 `ManagementWorkflow`
- `BaseNode`를 상속받은 `WebSearchNode`, `ResourceManagementNode`
- LangGraph의 `StateGraph`를 사용한 워크플로우 구성

## 1. 환경 설정

In [ ]:
# 환경 변수 및 경로 설정
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# 프로젝트 루트 디렉토리 찾기
project_root = Path.cwd()
if project_root.name != "Act1-Entertainment":
    # 상위 디렉토리에서 Act1-Entertainment 찾기
    for parent in project_root.parents:
        if parent.name == "Act1-Entertainment":
            project_root = parent
            break

# Python 경로에 프로젝트 루트 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# .env 파일 로드
env_path = project_root / ".env"
load_dotenv(env_path)

print(f"프로젝트 루트: {project_root}")
print(f"환경 변수 파일: {env_path}")
print(f"OPENAI_API_KEY 설정됨: {'예' if os.getenv('OPENAI_API_KEY') else '아니오'}")
print(f"TAVILY_API_KEY 설정됨: {'예' if os.getenv('TAVILY_API_KEY') else '아니오'}")

: 

In [ ]:
jupyter kernelspec list


: 

In [2]:
import sys
print(sys.executable)

c:\cursor\.venv\Scripts\python.exe


## 2. 프로젝트 구조 확인

In [2]:
# 기본 클래스 임포트
from agents.base_workflow import BaseWorkflow
from agents.base_node import BaseNode

# BaseWorkflow의 구조 확인
print("=== BaseWorkflow 구조 ===")
print(f"추상 메서드: {[method for method in dir(BaseWorkflow) if method.startswith('build')]}")
print(f"BaseWorkflow.__doc__:\n{BaseWorkflow.__doc__}")

print("\n=== BaseNode 구조 ===")
print(f"추상 메서드: {[method for method in dir(BaseNode) if method.startswith('execute')]}")
print(f"BaseNode.__doc__:\n{BaseNode.__doc__}")

ImportError: cannot import name 'BaseMetadata' from 'annotated_types' (unknown location)

## 3. Management 모듈 임포트

In [ ]:
# Management 워크플로우 관련 임포트
from agents.management.modules.state import ManagementState
from agents.management.modules.nodes import WebSearchNode, ResourceManagementNode
from agents.management.workflow import ManagementWorkflow, management_workflow

# 구조 확인
print("=== ManagementWorkflow 정보 ===")
print(f"부모 클래스: {ManagementWorkflow.__bases__}")
print(f"워크플로우 이름: {management_workflow.name}")
print(f"상태 클래스: {management_workflow.state}")

print("\n=== ManagementState 필드 ===")
print(ManagementState.__annotations__)

## 4. 노드 단위 테스트

In [ ]:
# WebSearchNode 단독 테스트
print("=== WebSearchNode 테스트 ===")

# 노드 인스턴스 생성 (verbose=True로 로깅 활성화)
web_search_node = WebSearchNode(verbose=True)
print(f"노드 이름: {web_search_node.name}")
print(f"BaseNode를 상속받음: {isinstance(web_search_node, BaseNode)}")

# 테스트용 상태 생성
test_state = ManagementState(
    project_id="TEST-001",
    request_type="resource_allocation",
    query="아이유 콘서트 기획",
    response=[],
    team_members=["기획팀", "마케팅팀"],
    resources_available={"budget": "10억원", "venue": "고척스카이돔"}
)

# execute 메서드 직접 호출
search_result = web_search_node.execute(test_state)
print(f"\n검색 결과 키: {search_result.keys()}")
print(f"검색 결과 내용: {search_result.get('search_results', 'N/A')[:200]}...")

In [ ]:
# ResourceManagementNode 단독 테스트
print("=== ResourceManagementNode 테스트 ===")

# 검색 결과를 상태에 추가
test_state["search_results"] = search_result.get("search_results", "검색 결과 없음")

# 노드 인스턴스 생성
resource_node = ResourceManagementNode(verbose=True)
print(f"노드 이름: {resource_node.name}")
print(f"BaseNode를 상속받음: {isinstance(resource_node, BaseNode)}")

# execute 메서드 호출
resource_result = resource_node.execute(test_state)
print(f"\n리소스 계획 결과 키: {resource_result.keys()}")
print(f"\n생성된 리소스 계획 (처음 500자):\n{resource_result['response'][:500]}...")

## 5. 워크플로우 빌드 및 구조 확인

In [ ]:
# 기존 인스턴스 사용 또는 새로 생성
print("=== ManagementWorkflow 빌드 ===")

# workflow.py에서 이미 생성된 인스턴스 사용
print(f"기존 워크플로우 인스턴스 사용: {management_workflow.name}")

# 워크플로우 빌드
compiled_workflow = management_workflow.build()
print(f"\n컴파일된 워크플로우 타입: {type(compiled_workflow)}")
print(f"워크플로우 이름: {compiled_workflow.name}")

# 그래프 구조 확인
graph = compiled_workflow.get_graph()
print(f"\n그래프 노드: {graph.nodes}")
print(f"그래프 엣지: {graph.edges}")

In [ ]:
# 그래프 시각화 (mermaid 다이어그램)
try:
    mermaid_graph = compiled_workflow.get_graph().draw_mermaid()
    print("=== 워크플로우 구조 (Mermaid) ===")
    print(mermaid_graph)
except Exception as e:
    print(f"Mermaid 다이어그램 생성 실패: {e}")

## 6. 전체 워크플로우 실행

In [ ]:
# 워크플로우 입력 데이터 준비
workflow_input = {
    "project_id": "IU-CONCERT-2024",
    "request_type": "resource_allocation",
    "query": "아이유 2024 연말 콘서트 기획",
    "response": [],
    "team_members": ["총괄 PD", "음향 감독", "조명 감독", "무대 디자이너", "보안팀장"],
    "resources_available": {
        "budget": "50억원",
        "venue": "고척스카이돔",
        "date_range": "2024년 12월 20-25일",
        "expected_audience": "20,000명/일"
    }
}

print("=== 워크플로우 실행 ===")
print(f"입력 데이터:")
for key, value in workflow_input.items():
    if key != "response":
        print(f"  {key}: {value}")

# invoke 메서드로 워크플로우 실행
result = compiled_workflow.invoke(workflow_input)

print("\n=== 실행 결과 ===")
print(f"검색 결과 존재: {'예' if result.get('search_results') else '아니오'}")
print(f"리소스 계획 생성: {'예' if result.get('response') else '아니오'}")
print(f"\n최종 응답:\n{result.get('response', '응답 없음')}")

## 7. 스트리밍 모드 실행

In [ ]:
# 스트리밍 모드로 워크플로우 실행 (각 노드의 실행 과정을 실시간으로 확인)
streaming_input = {
    "project_id": "IU-FANMEETING-2024",
    "request_type": "creator_development",
    "query": "아이유 글로벌 팬미팅 투어",
    "response": [],
    "team_members": ["이벤트 기획팀", "글로벌 마케팅팀", "통역팀"],
    "resources_available": {
        "budget": "30억원",
        "cities": ["서울", "도쿄", "상하이", "방콕", "싱가포르"],
        "duration": "2주"
    }
}

print("=== 스트리밍 모드 실행 ===")
print("각 노드의 실행 과정을 실시간으로 확인합니다.\n")

for event in compiled_workflow.stream(streaming_input):
    for node_name, node_output in event.items():
        print(f"\n[노드: {node_name}]")
        if isinstance(node_output, dict):
            for key, value in node_output.items():
                if key == "response" and value:
                    print(f"  - 응답 생성 완료 ({len(value)} 문자)")
                elif key == "search_results" and value:
                    print(f"  - 검색 완료: {str(value)[:100]}...")
                elif key not in ["response", "search_results"]:
                    print(f"  - {key}: {value}")

## 8. 다양한 시나리오 테스트

In [ ]:
# 여러 시나리오를 정의하고 테스트
scenarios = [
    {
        "name": "음악 앨범 제작",
        "input": {
            "project_id": "ALBUM-IU-2024",
            "request_type": "resource_allocation",
            "query": "아이유 정규 6집 앨범 제작",
            "response": [],
            "team_members": ["프로듀서", "작곡가", "편곡자", "녹음 엔지니어", "믹싱 엔지니어"],
            "resources_available": {
                "budget": "15억원",
                "studio": "서울 녹음실 A, B",
                "timeline": "4개월",
                "featuring_artists": ["Dean", "박재범"]
            }
        }
    },
    {
        "name": "드라마 OST 프로젝트",
        "input": {
            "project_id": "OST-DRAMA-2024",
            "request_type": "team_management",
            "query": "tvN 새 드라마 OST 제작팀 구성",
            "response": [],
            "team_members": ["음악감독", "OST 코디네이터"],
            "resources_available": {
                "budget": "5억원",
                "drama_title": "달의 연인",
                "episodes": "16부작",
                "required_songs": "10곡"
            }
        }
    }
]

for scenario in scenarios:
    print(f"\n{'='*60}")
    print(f"시나리오: {scenario['name']}")
    print(f"프로젝트 ID: {scenario['input']['project_id']}")
    print(f"요청 유형: {scenario['input']['request_type']}")
    print(f"쿼리: {scenario['input']['query']}")
    
    result = compiled_workflow.invoke(scenario['input'])
    
    print(f"\n결과 (처음 500자):")
    print(result.get('response', '응답 없음')[:500] + "...")

## 9. 에러 처리 및 디버깅

In [ ]:
# 필수 필드가 누락된 경우 테스트
print("=== 에러 처리 테스트 ===")

# 잘못된 입력 데이터 (필수 필드 누락)
invalid_input = {
    "project_id": "ERROR-TEST",
    # "request_type" 누락
    "query": "테스트",
    "response": []
}

try:
    result = compiled_workflow.invoke(invalid_input)
    print("워크플로우가 실행되었습니다.")
except Exception as e:
    print(f"예상된 에러 발생: {type(e).__name__}")
    print(f"에러 메시지: {str(e)}")

# API 키 확인
print("\n=== API 키 상태 ===")
print(f"OpenAI API 키: {'설정됨' if os.getenv('OPENAI_API_KEY') else '없음'}")
print(f"Tavily API 키: {'설정됨' if os.getenv('TAVILY_API_KEY') else '없음 (모의 검색 사용)'}")

## 10. 커스텀 워크플로우 생성 예제

In [ ]:
# 프로젝트 구조를 따라 새로운 워크플로우를 생성하는 예제
from langgraph.graph import StateGraph

class CustomManagementWorkflow(BaseWorkflow):
    """커스터마이징된 Management 워크플로우"""
    
    def __init__(self):
        super().__init__()
        self.state = ManagementState
    
    def build(self):
        """웹 검색을 2번 수행하는 커스텀 워크플로우"""
        builder = StateGraph(self.state)
        
        # 노드 추가
        builder.add_node("first_search", WebSearchNode())
        builder.add_node("second_search", WebSearchNode())  # 두 번째 검색
        builder.add_node("resource_planning", ResourceManagementNode())
        
        # 엣지 설정
        builder.set_entry_point("first_search")
        builder.add_edge("first_search", "second_search")
        builder.add_edge("second_search", "resource_planning")
        builder.set_finish_point("resource_planning")
        
        workflow = builder.compile()
        workflow.name = self.name
        return workflow

# 커스텀 워크플로우 테스트
print("=== 커스텀 워크플로우 테스트 ===")
custom_workflow = CustomManagementWorkflow()
custom_compiled = custom_workflow.build()

print(f"커스텀 워크플로우 이름: {custom_compiled.name}")
print(f"노드 개수: {len(custom_compiled.get_graph().nodes)}")

# 실행
custom_result = custom_compiled.invoke(workflow_input)
print(f"\n실행 완료. 응답 길이: {len(custom_result.get('response', ''))} 문자")

## 정리

이 노트북에서는 Act1-Entertainment 프로젝트의 기본 구조를 따라:

1. **BaseWorkflow**를 상속받은 ManagementWorkflow 구현
2. **BaseNode**를 상속받은 WebSearchNode, ResourceManagementNode 구현
3. LangGraph의 **StateGraph**를 사용한 워크플로우 구성
4. **invoke()** 및 **stream()** 메서드를 통한 워크플로우 실행
5. 다양한 시나리오 테스트 및 에러 처리

이 구조는 프로젝트의 다른 워크플로우(text, music, image 등)에도 동일하게 적용됩니다.